In [1]:
from lda_class_carol import LDA2

In [2]:
d1='a '*10
d2='b '*10
docs=[d1,d2]*10
a=LDA2(docs)
a.lda(2,5)

[[('a', 1.0), ('b', 0.0)], [('b', 1.0), ('a', 0.0)]]

In [5]:
d1 = "Java is a language for programming that develops a software for several platforms. A compiled code or bytecode on Java application can run on most of the operating systems including Linux, Mac operating system, and Linux. Most of the syntax of Java is derived from the C++ and C languages."
d2 = "Python supports multiple programming paradigms and comes up with a large standard library, paradigms included are object-oriented, imperative, functional and procedural."
d3 = "Go is typed statically compiled language. It was created by Robert Griesemer, Ken Thompson, and Rob Pike in 2009. This language offers garbage collection, concurrency of CSP-style, memory safety, and structural typing."
d4 = "A young girl when she first visited magical Underland, Alice Kingsleigh (Mia Wasikowska) is now a teenager with no memory of the place -- except in her dreams."
d5 = "Her life takes a turn for the unexpected when, at a garden party for her fiance and herself, she spots a certain white rabbit and tumbles down a hole after him. Reunited with her friends the Mad Hatter (Johnny Depp), the Cheshire Cat and others, Alice learns it is her destiny to end the Red Queen's (Helena Bonham Carter) reign of terror."
docs=[d1,d2,d3,d4,d5]
b=LDA2(docs)
b.lda(2,5,conv_threshold=1e-2,max_iter=100,niter=10)

[[('a', 0.050276621122145065),
  ('for', 0.04810886276255096),
  ('the', 0.036783318056150045),
  ('is', 0.03231381130513576),
  ('of', 0.029594672782335255)],
 [('and', 0.07158622751710034),
  ('a', 0.05297362474237385),
  ('the', 0.04497419621448641),
  ('of', 0.03221544351530179),
  ('her', 0.025758020506593097)]]

[array([31.22091129, 32.34822578]),
 array([16.40570138, 19.16343569]),
 array([ 6.99784544, 38.57129163]),
 array([11.87177   , 28.69736707]),
 array([33.27582362, 42.29331346])]